In [45]:
import re
import math
import sqlite3

In [6]:
#取出文档中的单词 并将单词全部变为小写
def getwords(doc):
    splitter = re.compile('\\W*')
    
    words = [s.lower() for s in splitter.split(doc) 
             if len(s) > 2 and len(s) < 20]
    
    return dict([(w , 1) for w in words])


#导入一些简单的训练数据
def sampletrain(c1):
    c1.train('Nobody owns the water' , 'good')
    c1.train('the quick rabbit jumps fences' , 'good')
    c1.train('buy pharmaceuticals now' , 'bad')
    c1.train('make quick money at the online casino' , 'bad')
    c1.train('the quick brown fox jumps' , 'good')

In [62]:
class classifier:
    def __init__(self , getfeatures , filename = None):
        self.fc = {} #统计特征 分类组合
        self.cc = {} #每个类别中文档数量
        self.getfeatures = getfeatures
        #add
        self.thresholds = {}
        
    #def incf(self , f , cat):
    #    self.fc.setdefault(f , {})
    #    self.fc[f].setdefault(cat , 0)
    #    self.fc[f][cat] += 1
    #
    #def incc(self , cat):
    #    self.cc.setdefault(cat , 0)
    #    self.cc[cat] += 1
    #    
    #def fcount(self , f , cat):
    #    if f in self.fc and cat in self.fc[f]:
    #        return float(self.fc[f][cat])
    #
    #    return 0.0
    #
    #def catcount(self , cat):
    #    if cat in self.cc:
    #        return float(self.cc[cat])
    #    
    #    return 0
    
    #def totalcount(self):
    #    return sum(self.cc.values())
    #
    #def categories(self):
    #    return self.cc.keys()
    
    #训练方法 目前就是对特征单词出现的次数进行统计
    def train(self , item , cat):
        features = self.getfeatures(item)
        
        for f in features:
            self.incf(f , cat)
        
        self.incc(cat)
        
        #下面这句话是在增加数据库操作之后添加的
        self.con.commit()
        
    #计算概率
    def fprob(self , f , cat):
        if self.catcount(cat) == 0:
            return 0
    
        return self.fcount(f , cat) / self.catcount(cat)
    
    def weightedprob(self , f , cat , prf , weight=1.0 , ap = 0.5):
        basicprob = prf(f , cat)
        
        totals = sum([self.fcount(f , c) for c in self.categories()])
        
        bp = ((weight*ap) + (totals*basicprob)) / (weight+totals)
        
        return bp
    
    def setthreshold(self , cat , t):
        self.thresholds[cat] = t
    
    def getthreshold(self , cat):
        if cat not in self.thresholds:
            return 1.0
        
        return self.thresholds[cat]
    
    def classify(self , item , default = None):
        probs = {}
        
        max = 0.0
        for cat in self.categories():
            probs[cat] = self.prob(item , cat)
            if probs[cat]>max:
                max = probs[cat]
                best = cat
            
        for cat in probs:
            if cat == best:
                continue
            if probs[cat]*self.getthreshold(best)>probs[best]:
                return default
        
        return best
    
    #===========
    #+++++++++++
    #植入数据库
    def setdb(self , dbfile):
        self.con = sqlite3.connect(dbfile)
        self.con.execute('create table if not exists fc(feature , category,count)')
        self.con.execute('create table if not exists cc(category , count)')
        
    def incf(self , f , cat):
        count = self.fcount(f , cat)
        if count == 0:
            self.con.execute("insert into fc values ('%s','%s',1)" % (f , cat))
        else:
            self.con.execute(
            "update fc set count=%d where feature='%s' and category='%s'" 
            % (count+1 , f , cat))
    
    def fcount(self , f ,cat):
        res = self.con.execute(
        "select count from fc where feature='%s' and category ='%s'" 
        % (f,cat)).fetchone()
        
        if res == None:
            return 0
        else:
            return float(res[0])
        
    def incc(self , cat):
        count = self.catcount(cat)
        if count == 0:
            self.con.execute(
            "insert into cc values ('%s' , 1)" % (cat))
        else:
            self.con.execute(
            "update cc set count=%d where category='%s'" % (count+1 , cat))
        
    def catcount(self , cat):
        res = self.con.execute(
        "select count from cc where category='%s'" % (cat)).fetchone()
        
        if res == None:
            return 0
        else:
            return float(res[0])
        
    def categories(self):
        cur = self.con.execute(
        "select category from cc")
        return [d[0] for d in cur]
    
    def totalcount(self):
        res = self.con.execute(
        "select sum(count) from cc").fetchone()
        
        if res == None:
            return 0
        
        return res[0]

In [4]:
c1 = classifier(getwords)
c1.train('the quick brown fox jumps over the lazy dog' , 'good')
c1.train('make quick money in the online casino' , 'bad')

c1.fcount('quick' , 'good')

1.0

In [5]:
c1.fcount('quick' , 'bad')

1.0

In [8]:
c1 = classifier(getwords)
sampletrain(c1)
c1.fprob('quick' , 'good')

0.6666666666666666

In [11]:
c1 = classifier(getwords)
sampletrain(c1)
c1.weightedprob('money' , 'good' , c1.fprob)

0.25

In [12]:
sampletrain(c1)
c1.weightedprob('money' , 'good' , c1.fprob)

0.16666666666666666

In [59]:
#naive bayes
class naivebayes(classifier):
    def docprob(self , item , cat):
        features = self.getfeatures(item)
        
        p = 1
        for f in features:
            p *= self.weightedprob(f , cat , self.fprob)
            
        return p
    
    def prob(self , item , cat):
        catprob = self.catcount(cat)/self.totalcount()
        docprob = self.docprob(item , cat)
        
        return docprob * catprob
    
    

In [15]:
c1 = naivebayes(getwords)
sampletrain(c1)

c1.prob('quick rabbit' , 'good')


0.15624999999999997

In [16]:
c1.prob('quick rabbit' , 'bad')

0.05

In [25]:
c1 = naivebayes(getwords)
sampletrain(c1)

c1.classify('quick rabbit' , default = 'unknown')

'good'

In [26]:
c1.classify('quick money' , default='unknown')

'bad'

In [27]:
c1.setthreshold('bad' , 3.0)
c1.classify('quick money' , default='unknown')

'unknown'

In [28]:
for i in range(10):
    sampletrain(c1)
    
c1.classify('quick money' , default='unknown')

'bad'

In [67]:
#费舍尔方法
class fisherclassifier(classifier):
    def cprob(self , f , cat):
        clf = self.fprob(f , cat)
        
        if clf == 0:
            return 0
        
        freqsum = sum([self.fprob(f , c) for c in self.categories()])
        
        p = clf/(freqsum)
        
        return p
    
    def fisherprob(self , item , cat):
        p = 1
        
        features = self.getfeatures(item)
        
        for f in features:
            p*= (self.weightedprob(f , cat , self.cprob))
        
        fscore = -2*math.log(p)
        
        return self.invchi2(fscore , len(features)*2)
    
    #倒置对数卡方函数
    def invchi2(self , chi , df):
        m = chi/2.0
        
        sum_ = term = math.exp(-m)
        
        for i in range(1 , df//2):
            term *= m/i
            sum_ += term
        
        return min(sum_ , 1.0)
    
    def __init__(self , getfeatures):
        classifier.__init__(self , getfeatures)
        self.minimums = {}
        
    
    def setminimum(self , cat , min_):
        self.minimums[cat] = min_
        
    def getminimum(self , cat):
        if cat not in self.minimums:
            return 0
        
        return self.minimums[cat]
    
    def classify(self , item , default = None):
        best = default
        max_ = 0.0
        
        for c in self.categories():
            p = self.fisherprob(item , c)
            
            if p>self.getminimum(c) and p>max_:
                best = c
                max_ = p
        
        return best

In [33]:
c1 = fisherclassifier(getwords)
sampletrain(c1)
c1.cprob('quick' , 'good')

0.5714285714285715

In [34]:
c1.cprob('money' , 'bad')

1.0

In [35]:
c1.weightedprob('money','bad' , c1.cprob)

0.75

In [38]:
c1 = fisherclassifier(getwords)
sampletrain(c1)
c1.cprob('quick' , 'good')

0.5714285714285715

In [39]:
c1.fisherprob('quick rabbit' , 'good')

0.78013986588958

In [40]:
c1.fisherprob('quick rabbit' , 'bad')

0.3563359628333526

In [43]:
c1 = fisherclassifier(getwords)
sampletrain(c1)
c1.classify('quick rabbit')

'good'

In [44]:
c1.classify('quick money')

'bad'

In [69]:
c1 = fisherclassifier(getwords)
c1.setdb('test1.db')
sampletrain(c1)

c2 = naivebayes(getwords)
c2.setdb('test1.db')
c2.classify('quick money')

u'bad'